In [1]:
import sys 
parent_directory = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago'
sys.path.append(parent_directory)
search_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/local_search'
sys.path.append(search_path)
travel_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/traveltime.py'
sys.path.append(travel_path)
load_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/load_data.py'
sys.path.append(load_path)
plot_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/data_prep/plot.py'
sys.path.append(plot_path)

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geojson
import json
import plotly.express as px
import matplotlib as plt
from gerrychain import Graph
import networkx as nx
import random
import pickle

import tools.load_data as load
import tools.plot as plot
import tools.traveltime as travel

In [ ]:
"""import nbimporter as nbim


def data_processing():
    # Define your function here
    pass


#Constructs a common Python idiom that prevents certain code from being executed when the script is imported as a module in another script. 
#The code block under this if-statement will only run if the script is executed directly (not when imported).


if __name__ == "__main__":
    # Code inside this block won't run if this notebook is imported using nbimporter
    data_processing()
    print("This prints only if the notebook is executed directly, not when imported.")
    



from load_data import load_and_preprocess_data, load_cached_data, cache_data

cache_path = 'data_cache.pkl'
try:
    # Attempt to load cached data
    data = load_cached_data(cache_path)
    print("Loaded data from cache.")
except (FileNotFoundError, IOError):
    # Load and preprocess data if cache is not available
    data = load_and_preprocess_data('your_data.csv')
    cache_data(data, cache_path)
    print("Loaded data from source and cached it.")

# Now 'data' is ready to use """

In [3]:
chicago = gpd.read_file('data/chicago.json')
center_lat, center_lon = load.projection(chicago)  # see gerrychain library for that function
fig =  plot.fig_blocks(chicago)

/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/plot.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center={"lat": data.geometry.centroid.y.mean(), "lon": data.geometry.centroid.x.mean()},


# Dual Graph of Census Blocks

In [4]:
""" We use Gerrychain library to construct a dual graph of Census blocks. """

#graph = Graph.from_geodataframe(chicago)
#nx.draw(graph,pos={node:(graph.nodes()[node]['centroid'].x,graph.nodes()[node]['centroid'].y) for node in graph.nodes()}, node_size=20)
#graph.to_json("GraphToJson.json")

graph = Graph.from_json("data/GraphToJson.json")

# Opening JSON file
#f = open('GraphToJson.json')
# returns JSON object as a dictionary
#data = json.load(f)
#f.close()
#data.keys()

"""adjacency = random.choice(list(data['adjacency']))
adjacency"""

"""node = random.choice(list(graph.nodes))
graph.nodes[node]['GEOID20'] """

"""first_node = list(graph.nodes)[0]
node_attributes = graph.nodes[first_node]
print(f"Attributes for node {first_node}: {node_attributes}")"""

""" lookup()  #Lookup a node/field attribute. Returns the value of the attribute field at node.""" 

' lookup()  #Lookup a node/field attribute. Returns the value of the attribute field at node.'

- The areas of the polygons are included as node attributes (with key `area`).
        
- The shared perimeter of neighboring polygons are included as edge attributes (with key `shared_perim`).

- Nodes corresponding to polygons on the boundary of the union of all the geometries (e.g., the state, if your dataframe describes VTDs) have a    `boundary_node` attribute (set to `True`) and a `boundary_perim` attribute
with the length of this "exterior" boundary.

- Gerrychain library has centroid function.

# Get Populations from DHC Data

In [5]:
dhc = pd.read_csv('data/dhc_2020.csv')
pop_dict = load.get_column(data = dhc, column = 'TOT_POP')  # returns {key: geoid, value: population}
load.add_to_graph(graph, pop_dict)  # adds 'population' attributes to nodes
load.add_to_data(chicago, pop_dict)

See in API that

classgerrychain.partition.GeographicPartition

crosses_parts(edge: Tuple)→ bool

flip(flips: Dict)→ Partition

plot(geometries=None, **kwargs)

# Generate Random PHCs

In [7]:
#phc_df, selected_blocks = load.random_phc(chicago)
#candidates = load.add_phc(chicago, graph, selected_blocks)
#fig = plot.add_phc(fig, phc_df)
#fig.show()

In [ ]:
"Move tree from local_search to tree.py"
#from local_search import tree
#uniform = tree.uniform_spanning_tree(graph)
#nx.set_edge_attributes(graph, 10, name='time')

# Calculate Travel Times

In [8]:
"travel_times alculated and saved."

#for u, v in graph.edges():
#    graph[u][v]['time'] = 10
#travel_times = travel.travel_time(graph, candidates)

# Save Objects

In [16]:
" 1) Graph is saved as graph.pkl in prepared_data folder."
# load.cache_data(graph, file_path='prepared_data/graph.pkl', method = 'pickle')  
" Surprisingly, it uses only 10MB memory, while GraphToJson.json uses 27MB."

' Surprisingly, it uses only 10MB memory, while GraphToJson.json uses 27MB.'

In [12]:
""" 2) Travel times is saved as travel_times.pkl.gz
    ujson is loaded in 35 sec, and uses 140MB 
    pkl.gz is loaded in 16-17 sec, and uses 130MB memory. """
# load.cache_data(travel_times, file_path='prepared_data/travel_times.pkl.gz', method = 'pickle_gzip')

In [17]:
""" 3) Chicago data is saved as chicago.pkl.
    pkl is loaded in 1 sec, and uses 19MB memory. 
    Original chicago.json file uses 38MB.
    I didn't really get it. pkl is better than json in both memory and loading."""
    
# load.cache_data(chicago, file_path='prepared_data/chicago.pkl', method='pickle')

In [18]:
""" 4) Possible PHC locations (candidates) are saved as candidates.pkl"""
# load.cache_data(candidates, file_path='prepared_data/candidates.pkl', method='pickle')